In [47]:
# quickly search 'model annotations' for reactoms; if does not exist, forget aboout it??
# developing (and testing) Alternative methods for species
import collections
import compress_pickle
import editdistance
import itertools
import libsbml
import numpy as np
import os
import pickle
import pandas as pd
import re
import sys
import time
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
REACTOME_DIR = os.path.join(DATA_DIR, 'reactome')
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'

# dir for alternative methods for species
ALT_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/AMAS_suppl/alt_methods_species_data'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import recommender
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools

In [13]:
biomd_fnames = [val for val in os.listdir(BIOMODEL_DIR) if val[-4:]=='.xml']
biomd_fpaths = [os.path.join(BIOMODEL_DIR, val) for val in biomd_fnames]

In [29]:
models_with_reactome = []
reactome_dict = dict()
for idx, one_biomd in enumerate(biomd_fnames):
  if idx % 100 == 0:
    print("We are at", idx)
  one_fpath = os.path.join(BIOMODEL_DIR, one_biomd)
  reader = libsbml.SBMLReader()
  document = reader.readSBML(one_fpath)
  model = document.getModel()
  model_anot = model.getAnnotationString()
  if model_anot:
    ont_anot = tools.getOntologyFromString(string_annotation=model_anot)
    if any([val[0].lower()=='reactome' for val in ont_anot]):
      models_with_reactome.append(one_biomd)
      reactome_dict[one_biomd] = ont_anot

We are at 0
We are at 100
We are at 200
We are at 300
We are at 400
We are at 500
We are at 600
We are at 700
We are at 800
We are at 900


In [94]:
len(reactome_dict)

26

In [34]:
model_reactome_chebi = []
chebi_dict = dict()
for idx, one_biomd in enumerate(models_with_reactome):
  one_fpath = os.path.join(BIOMODEL_DIR, one_biomd)
  recom = recommender.Recommender(libsbml_fpath=one_fpath)
  if recom.species.exist_annotation:
    model_reactome_chebi.append(one_biomd)
    chebi_dict[one_biomd] = recom.species.exist_annotation

In [37]:
sub_reactome_dict = {val:reactome_dict[val] for val in chebi_dict.keys()}

In [97]:
print(len(sub_reactome_dict))
sub_reactome_dict

7


{'BIOMD0000000255.xml': [('reactome', 'REACT_9417.3'), ('go', 'GO:0007173')],
 'BIOMD0000000691.xml': [('reactome', 'R-HSA-70171'),
  ('go', 'GO:0006096'),
  ('kegg.pathway', 'map00010')],
 'BIOMD0000000467.xml': [('reactome', 'REACT_12443.3'),
  ('go', 'GO:0006809'),
  ('go', 'GO:0002537')],
 'BIOMD0000000465.xml': [('reactome', 'REACT_12443.3'),
  ('go', 'GO:0034405'),
  ('go', 'GO:0006809'),
  ('go', 'GO:0002537')],
 'BIOMD0000000303.xml': [('reactome', 'REACT_6932.3'), ('go', 'GO:0006956')],
 'BIOMD0000000015.xml': [('kegg.pathway', 'hsa00230'),
  ('reactome', 'REACT_522'),
  ('go', 'GO:0006144')],
 'BIOMD0000000188.xml': [('reactome', 'REACT_309'), ('go', 'GO:0043516')]}

In [99]:
recom = recommender.Recommender(libsbml_fpath=os.path.join(BIOMODEL_DIR, 'BIOMD0000000691.xml'))
recom.species.exist_annotation

{'S1__Cell_1_': ['CHEBI:17234'],
 'S1__Cell_2_': ['CHEBI:17234'],
 'S3__Cell_1_': ['CHEBI:89363'],
 'S3__Cell_2_': ['CHEBI:89363'],
 'N2__Cell_1_': ['CHEBI:16908'],
 'N2__Cell_2_': ['CHEBI:16908'],
 'A3__Cell_1_': ['CHEBI:15422'],
 'A3__Cell_2_': ['CHEBI:15422'],
 'N1__Cell_1_': ['CHEBI:15846'],
 'N1__Cell_2_': ['CHEBI:15846'],
 'A2__Cell_1_': ['CHEBI:16761'],
 'A2__Cell_2_': ['CHEBI:16761']}

In [107]:
set((itertools.chain(*[val[1] for val in recom.species.exist_annotation_formula.items()])))

{'C10N5O10P2', 'C10N5O13P3', 'C21N7O14P2', 'C3O10P2', 'C6O6'}

In [46]:
chebi_model691 = list(set(list(itertools.chain(*[val[1] \
                                                 for val in chebi_dict['BIOMD0000000691.xml'].items()]))))
chebi_model691

['CHEBI:17234',
 'CHEBI:15422',
 'CHEBI:16761',
 'CHEBI:16908',
 'CHEBI:89363',
 'CHEBI:15846']

In [48]:
with open(os.path.join(REACTOME_DIR, 'reactome2chebi.lzma'), 'rb') as f:
  reactome2chebi = compress_pickle.load(f)

In [51]:
one_k = list(reactome2chebi.keys())[0]
reactome2chebi[one_k].intersection(chebi_model691)

set()

In [53]:
match_props = []
for one_k in reactome2chebi.keys():
  match_props.append((one_k, len(reactome2chebi[one_k].intersection(chebi_model691)) / 
                      len(reactome2chebi[one_k])))

In [92]:
len(reactome2chebi)

19276

In [69]:
# pure chebi comparison didn't yield anything :(((
np.max([val[1] for val in match_props])

0.0

In [67]:
one_set_reactome = reactome2chebi['R-HSA-70171']
one_list_reactome = set([cn.REF_CHEBI2FORMULA[val] for val in one_set_reactome])
one_list_query = set([cn.REF_CHEBI2FORMULA[val] for val in chebi_model691])

In [68]:
len(one_list_reactome.intersection(one_list_query)) / len(one_list_reactome)

0.23809523809523808

In [72]:
# Re-do using reactome2formula
reactome2formula = dict()
for one_k in reactome2chebi.keys():
  one_set_reactome = reactome2chebi[one_k]
  reactome2formula[one_k] = set([cn.REF_CHEBI2FORMULA[val] for val in one_set_reactome \
                                if val in cn.REF_CHEBI2FORMULA.keys()])

In [85]:
new_match_props = []
for one_k in reactome2formula.keys():
  num_match = len(reactome2formula[one_k].intersection(one_list_query))
  num_pathway_comps = len(reactome2formula[one_k])
  if num_pathway_comps == 0:
    continue
  num_prop = num_match / num_pathway_comps
  new_match_props.append((one_k, num_match, num_prop))

In [83]:
reactome2chebi['R-HSA-5657562']

{'CHEBI:15824', 'CHEBI:30616'}

In [108]:
# match rule; find the matximum matches, and detect max proportions from them.. continue
max_num_match = np.max([val[1] for val in new_match_props])
max_match_props = [val for val in new_match_props if val[1]==max_num_match]
from operator import itemgetter
res = sorted(max_match_props,key=itemgetter(2), reverse=True)

In [117]:
res_df = pd.DataFrame(res, columns=['ID', 'Number of Matches', 'Proportion of Matches'])
res_df.iloc[:29, :]

,ID,Number of Matches,Proportion of Matches
0,R-PFA-70171,5,0.312500
1,R-SCE-70171,5,0.263158
2,R-SPO-70171,5,0.263158
3,R-DDI-70171,5,0.250000
4,R-PFA-70326,5,0.250000
5,R-BTA-70171,5,0.238095
6,R-CEL-70171,5,0.238095
7,R-CFA-70171,5,0.238095
8,R-DME-70171,5,0.238095
9,R-DRE-70171,5,0.238095


In [88]:
from operator import itemgetter
sorted(new_match_props,key=itemgetter(1), reverse=True)

[('R-BTA-1430728', 5, 0.007194244604316547),
 ('R-CFA-1430728', 5, 0.007407407407407408),
 ('R-DME-1430728', 5, 0.008944543828264758),
 ('R-DRE-1430728', 5, 0.00715307582260372),
 ('R-HSA-1430728', 5, 0.006587615283267457),
 ('R-MMU-1430728', 5, 0.007052186177715092),
 ('R-RNO-1430728', 5, 0.007142857142857143),
 ('R-SSC-1430728', 5, 0.0072254335260115606),
 ('R-XTR-1430728', 5, 0.007342143906020558),
 ('R-CEL-1430728', 5, 0.009191176470588236),
 ('R-DDI-1430728', 5, 0.009523809523809525),
 ('R-GGA-1430728', 5, 0.007374631268436578),
 ('R-SCE-1430728', 5, 0.013192612137203167),
 ('R-SPO-1430728', 5, 0.013440860215053764),
 ('R-PFA-1430728', 5, 0.018050541516245487),
 ('R-BTA-71387', 5, 0.05263157894736842),
 ('R-CEL-71387', 5, 0.06493506493506493),
 ('R-CFA-71387', 5, 0.053763440860215055),
 ('R-DDI-71387', 5, 0.06756756756756757),
 ('R-DME-71387', 5, 0.06097560975609756),
 ('R-DRE-71387', 5, 0.05263157894736842),
 ('R-GGA-71387', 5, 0.05555555555555555),
 ('R-HSA-71387', 5, 0.05263157